In [1]:
import geopandas as gpd
import pandas as pd
import verde as vd
import numpy as np
from shapely import geometry
from sources import source_code as f

import matplotlib.pyplot as plt
%matplotlib widget

gdb='/home/ggrl/geodatabase/'

In [92]:
rparaim_litologia_100k = f.importar('litologia_100k','Rio Paraim')

In [56]:
rparaim_litologia_100k.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
dic_cartas, dic_raw_meta = f.get_region('100k','SC23_Z_A_IV','gama_1089')

importando dados geofisicos
# --- Iniciando seleção de área de estudo
1 folha cartográfica selecionada

Listando atributos dos dados geofisicos
# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']
lista de atributos geograficos = ['LONGITUDE', 'LATITUDE']
lista de atributos projetados = ['X', 'Y']


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)



# --- Início da iteração entre as folhas cartográficas #
# Folha de código: SC23_Z_A_IV
 Atualizando dados brutos em dic_cartas['raw_data']
 com 81859 pontos


In [6]:
rparaim_litologia_100k.plot('SIGLA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [59]:
data = dic_cartas['raw_data']['SC23_Z_A_IV']

In [60]:
chain_list = ['CTCOR', 'KPERC', 'MDT', 'eU', 'eth']

grids = {'CTCOR':(),'KPERC':(), 'MDT':(), 'eU':(), 'eth':()}

coordinates = (data.X.values, data.Y.values)

for i in chain_list:
    print(i)
    
    chain = vd.Chain([
        ('trend', vd.Trend(degree=1)),
        ('reduce', vd.BlockReduce(np.median, spacing=1000)),
        ('spline', vd.Spline())
    ])
    
    chain.fit(coordinates, data[i])
    
    grid = chain.grid(spacing=100, data_names=[i],pixel_register=True)
    grids[i] = vd.distance_mask(coordinates, maxdist=1000, grid= grid)

CTCOR
KPERC
MDT
eU
eth


In [61]:
data = list()
for i in chain_list:
    df = grids[i].to_dataframe()
    data.append(df[i])

In [62]:
geof = pd.concat(data,axis=1, join='inner')

In [63]:
geof

CTCOR     KPERC         MDT        eU  \
northing     easting                                                      
8.784026e+06 500055.290814  1664.763803  1.189200  514.194082  2.095518   
             500155.272441  1673.819919  1.214216  513.882732  2.097788   
             500255.254068  1681.159851  1.238521  513.385165  2.098775   
             500355.235695  1686.161779  1.261792  512.624688  2.098343   
             500455.217322  1688.107483  1.283635  511.512891  2.096363   
...                                 ...       ...         ...       ...   
8.839255e+06 554245.332678  1135.953136  0.741216  475.146335  1.638301   
             554345.314305  1140.910466  0.745416  473.820217  1.697407   
             554445.295932  1146.001066  0.749709  472.571602  1.753559   
             554545.277559  1151.177974  0.754013  471.430130  1.805485   
             554645.259186  1156.397331  0.758248  470.417979  1.852235   

                                  eth  
northing     easting                   
8.784026e+06 500055.290814  16.506882  
             500155.272441  16.544889  
             500255.254068  16.565282  
             500355.235695  16.561460  
             500455.217322  16.525815  
...                               ...  
8.839255e+06 554245.332678  12.101631  
             554345.314305  12.068844  
             554445.295932  12.040976  
             554545.277559  12.019225  
             554645.259186  12.004427  

[302491 rows x 5 columns]

In [65]:
geof.reset_index(drop=True,inplace=True)
geof

,index,northing,easting,CTCOR,KPERC,MDT,eU,eth
0,0,8.784026e+06,500055.290814,1664.763803,1.189200,514.194082,2.095518,16.506882
1,1,8.784026e+06,500155.272441,1673.819919,1.214216,513.882732,2.097788,16.544889
2,2,8.784026e+06,500255.254068,1681.159851,1.238521,513.385165,2.098775,16.565282
3,3,8.784026e+06,500355.235695,1686.161779,1.261792,512.624688,2.098343,16.561460
4,4,8.784026e+06,500455.217322,1688.107483,1.283635,511.512891,2.096363,16.525815
...,...,...,...,...,...,...,...,...
302486,302486,8.839255e+06,554245.332678,1135.953136,0.741216,475.146335,1.638301,12.101631
302487,302487,8.839255e+06,554345.314305,1140.910466,0.745416,473.820217,1.697407,12.068844
302488,302488,8.839255e+06,554445.295932,1146.001066,0.749709,472.571602,1.753559,12.040976
302489,302489,8.839255e+06,554545.277559,1151.177974,0.754013,471.430130,1.805485,12.019225


In [66]:
geof['geometry'] = [geometry.Point(x,y) for x, y in zip(geof['easting'], geof['northing'])]

In [67]:
gdf = gpd.GeoDataFrame(geof,crs=32723)
gdf = gdf.set_crs(32723, allow_override=True)

In [68]:
gdf.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [69]:
gdf

,index,northing,easting,CTCOR,KPERC,MDT,eU,eth,geometry
0,0,8.784026e+06,500055.290814,1664.763803,1.189200,514.194082,2.095518,16.506882,POINT (500055.291 8784025.966)
1,1,8.784026e+06,500155.272441,1673.819919,1.214216,513.882732,2.097788,16.544889,POINT (500155.272 8784025.966)
2,2,8.784026e+06,500255.254068,1681.159851,1.238521,513.385165,2.098775,16.565282,POINT (500255.254 8784025.966)
3,3,8.784026e+06,500355.235695,1686.161779,1.261792,512.624688,2.098343,16.561460,POINT (500355.236 8784025.966)
4,4,8.784026e+06,500455.217322,1688.107483,1.283635,511.512891,2.096363,16.525815,POINT (500455.217 8784025.966)
...,...,...,...,...,...,...,...,...,...
302486,302486,8.839255e+06,554245.332678,1135.953136,0.741216,475.146335,1.638301,12.101631,POINT (554245.333 8839254.694)
302487,302487,8.839255e+06,554345.314305,1140.910466,0.745416,473.820217,1.697407,12.068844,POINT (554345.314 8839254.694)
302488,302488,8.839255e+06,554445.295932,1146.001066,0.749709,472.571602,1.753559,12.040976,POINT (554445.296 8839254.694)
302489,302489,8.839255e+06,554545.277559,1151.177974,0.754013,471.430130,1.805485,12.019225,POINT (554545.278 8839254.694)


In [94]:
rparaim_litologia_100k.to_crs(32723,inplace=True)
rparaim_litologia_100k.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [72]:
base = rparaim_litologia_100k.plot('SIGLA')
gdf.plot(c=gdf.MDT,ax=base,markersize=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [73]:
gdf

,index,northing,easting,CTCOR,KPERC,MDT,eU,eth,geometry
0,0,8.784026e+06,500055.290814,1664.763803,1.189200,514.194082,2.095518,16.506882,POINT (500055.291 8784025.966)
1,1,8.784026e+06,500155.272441,1673.819919,1.214216,513.882732,2.097788,16.544889,POINT (500155.272 8784025.966)
2,2,8.784026e+06,500255.254068,1681.159851,1.238521,513.385165,2.098775,16.565282,POINT (500255.254 8784025.966)
3,3,8.784026e+06,500355.235695,1686.161779,1.261792,512.624688,2.098343,16.561460,POINT (500355.236 8784025.966)
4,4,8.784026e+06,500455.217322,1688.107483,1.283635,511.512891,2.096363,16.525815,POINT (500455.217 8784025.966)
...,...,...,...,...,...,...,...,...,...
302486,302486,8.839255e+06,554245.332678,1135.953136,0.741216,475.146335,1.638301,12.101631,POINT (554245.333 8839254.694)
302487,302487,8.839255e+06,554345.314305,1140.910466,0.745416,473.820217,1.697407,12.068844,POINT (554345.314 8839254.694)
302488,302488,8.839255e+06,554445.295932,1146.001066,0.749709,472.571602,1.753559,12.040976,POINT (554445.296 8839254.694)
302489,302489,8.839255e+06,554545.277559,1151.177974,0.754013,471.430130,1.805485,12.019225,POINT (554545.278 8839254.694)


In [74]:
gdf.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [95]:
rparaim_litologia_100k.reset_index(drop=True,inplace=True)

In [96]:
rparaim_litologia_100k

,ID_UNIDADE,SIGLA,HIERARQUIA,NOME,AMBIENTE_T,SUB_AMBIEN,SIGLA_PAI,NOME_PAI,LEGENDA,ESCALA,...,ERA_MIN,ERA_MAX,SISTEMA_MI,SISTEMA_MA,EPOCA_MIN,EPOCA_MAX,SIGLAS_HIS,SHAPE_AREA,SHAPE_LEN,geometry
0,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,1:100.000,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.000041,0.037400,"POLYGON ((500176.894 8821390.751, 500170.654 8..."
1,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,1:100.000,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.001087,0.257896,"POLYGON ((505306.762 8803062.104, 505258.182 8..."
2,4873,NP1rp3,Unidade,"Unidade Rio Preto, xisto",None,None,NP1rp,NP1rp - Grupo Rio Preto,Muscovita-xistos e quartzo-xistos predominante...,1:100.000,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"MP3NP1rp3, M3N1rp1x, MP3NP1rp1x",0.000170,0.094417,"POLYGON ((505145.795 8801235.292, 506323.529 8..."
3,4876,NP1rp2q,Litofácies,"Litofácies Rio Preto, metassedimentos quartzos...",None,None,MP3NP1rp2,"MP3NP1rp2 - Unidade Rio Preto, metassedimentos...",Quartzitos a muscovita-quartzitos puros a impu...,1:100.000,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"M3N1rp2q, MP3NP1rp2q",0.000015,0.020423,"POLYGON ((509741.424 8784951.753, 509758.753 8..."
4,5773,PP2cmv,Unidade,"Unidade Cristalândia do Piauí, metamáfica",None,None,A4PPc,A4PPc - Complexo Cristalândia do Piauí,"Corpos gabróicos a metagabros, compostos por a...",1:100.000,...,Paleoproterozóico,Paleoproterozóico,Riaciano,Riaciano,Série 3,Série 3,None,0.000323,0.077072,"POLYGON ((510857.857 8811448.300, 510816.261 8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,4876,NP1rp2q,Litofácies,"Litofácies Rio Preto, metassedimentos quartzos...",None,None,MP3NP1rp2,"MP3NP1rp2 - Unidade Rio Preto, metassedimentos...",Quartzitos a muscovita-quartzitos puros a impu...,1:100.000,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"M3N1rp2q, MP3NP1rp2q",0.000045,0.063783,"POLYGON ((548787.138 8829535.408, 548759.773 8..."
137,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,1:100.000,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.012576,0.986684,"POLYGON ((543318.427 8837536.690, 543374.133 8..."
138,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,1:100.000,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.043917,1.961410,"POLYGON ((554660.641 8807055.364, 554656.953 8..."
139,4873,NP1rp3,Unidade,"Unidade Rio Preto, xisto",None,None,NP1rp,NP1rp - Grupo Rio Preto,Muscovita-xistos e quartzo-xistos predominante...,1:100.000,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"MP3NP1rp3, M3N1rp1x, MP3NP1rp1x",0.023544,3.651426,"POLYGON ((554702.380 8832876.999, 554702.379 8..."


In [97]:
rparaim_litologia_100k['SIGLA'].iloc[rparaim_litologia_100k.distance(gdf.geometry[0]).idxmin()]

'NP1rp3'

In [106]:
rparaim_litologia_100k.intersects(gdf)

/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/base.py:32: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


0         False
1         False
2         False
3         False
4         False
          ...  
302486    False
302487    False
302488    False
302489    False
302490    False
Length: 302491, dtype: bool

In [115]:
a = rparaim_litologia_100k[rparaim_litologia_100k.touches(gdf)]
b = gdf[gdf.touches(rparaim_litologia_100k)]

/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/base.py:32: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")
/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1299: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)
/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/base.py:32: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [127]:
a = rparaim_litologia_100k[rparaim_litologia_100k.within(gdf)]
b = gdf[gdf.within(rparaim_litologia_100k)]

/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/base.py:32: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")
/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1299: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)
/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/base.py:32: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [128]:
a

,ID_UNIDADE,SIGLA,HIERARQUIA,NOME,AMBIENTE_T,SUB_AMBIEN,SIGLA_PAI,NOME_PAI,LEGENDA,ESCALA,...,ERA_MIN,ERA_MAX,SISTEMA_MI,SISTEMA_MA,EPOCA_MIN,EPOCA_MAX,SIGLAS_HIS,SHAPE_AREA,SHAPE_LEN,geometry


In [129]:
b

,index,northing,easting,CTCOR,KPERC,MDT,eU,eth,geometry
68,68,8.784026e+06,506854.041453,1103.356289,0.674370,492.977945,1.513783,12.201865,POINT (506854.041 8784025.966)
93,93,8.784026e+06,509353.582130,857.289124,0.466327,500.879340,1.152214,11.304832,POINT (509353.582 8784025.966)


In [141]:
geof['closest_unid'] = gdf['geometry'].apply(lambda x: rparaim['SIGLA'].iloc[rparaim.distance(x).idxmin()]) # .idxmin() Retorna o indice do menor valor 

In [142]:
geof.to_csv(gdb+'')

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry,closest_unid
0,500105.462198,8.784085e+06,1659.760751,1.120279,514.907207,2.018264,16.542191,POINT (500105.462 8784085.083),NP1rp3
1,500105.462198,8.784285e+06,1632.120183,1.106715,513.074245,2.022793,16.315067,POINT (500105.462 8784284.789),NP1rp3
2,500105.462198,8.784484e+06,1591.655859,1.087899,509.603425,2.028829,15.941735,POINT (500105.462 8784484.495),NP1rp3
3,500105.462198,8.784684e+06,1537.350905,1.064791,504.364548,2.034102,15.410646,POINT (500105.462 8784684.201),NP1rp3
4,500105.462198,8.784884e+06,1472.429274,1.039333,497.970216,2.035450,14.764774,POINT (500105.462 8784883.907),NP1rp3
...,...,...,...,...,...,...,...,...,...
75616,554593.697802,8.838405e+06,1164.247622,0.729238,487.197021,2.040467,11.906200,POINT (554593.698 8838405.093),NP1rp3
75617,554593.697802,8.838605e+06,1167.331706,0.755726,481.847074,2.063674,11.927215,POINT (554593.698 8838604.799),NP1rp3
75618,554593.697802,8.838805e+06,1169.905155,0.778491,477.275848,2.058947,11.981648,POINT (554593.698 8838804.505),NP1rp3
75619,554593.697802,8.839004e+06,1170.249431,0.793855,473.794391,2.015057,12.065598,POINT (554593.698 8839004.211),NP1rp3


In [151]:
rparaim = rparaim.to_crs(4326)

In [ ]:
l_1kk = mf.litologia('l_1kk')

In [153]:
base = l_1kk.plot('SIGLA')
rparaim.boundary.plot(ax=base)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [173]:
plt.figure('Terrain')
grids['MDT'].MDT.plot(cmap='terrain')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(500005.29999999993, 554693.8599999999, 8783985.23, 8839303.77)

In [174]:
plt.figure('Contagem Total')
grids['CTCOR'].CTCOR.plot(cmap='hsv')
plt.axis('scaled')

<ipython-input-174-d2c99bc4b4fe>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure('Contagem Total')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(500005.29999999993, 554693.8599999999, 8783985.23, 8839303.77)